In [1]:
from selenium import webdriver
import time
import pandas as pd

In [2]:
# 데이터를 저장할 딕셔너리
data_dict = {
    "date" : [],
    "team1_name" : [],
    "team2_name" : [],
    "team1_point" : [],
    "team2_point" : []
}

# 웹 드라이버 로딩
driver = webdriver.Chrome()

In [3]:
# 페이지를 요청하기
driver.get('https://sports.news.naver.com/kbaseball/schedule/index.nhn')

### 날짜를 입력받아 해당 날짜의 데이터를 추출한다.

In [4]:
def getData(script_code) :
    # 스크립트를 실행시켜 원하는 날짜가 나오도록 한다.
    driver.execute_script(script_code)
    time.sleep(1)
    # 스케쥴 표 전체를 가져온다.
    sch_score_tag = driver.find_element_by_css_selector('#sch_recent_scoreboard > div > div')
    
    # 해당 날짜에 경기가 있는지 확인한다.
    temp_tag = sch_score_tag.find_elements_by_css_selector('.banner_mon')
    # 스크립트 코드에서 날짜만 추출한다.
    date_str = script_code.split("'")[1]
    
    # 경기가 없다면...
    if len(temp_tag) > 0 :
        print(f'{date_str} : 경기없음')
    else :
        print(f'{date_str} : 경기있음')
        li_list = driver.find_elements_by_css_selector('#todaySchedule > li.end')
        for li_tag in li_list : 
            team1_name = li_tag.find_element_by_css_selector('div.vs_lft > p > strong')
            team2_name = li_tag.find_element_by_css_selector('div.vs_rgt > p > strong')
            team1_point = li_tag.find_element_by_css_selector('div.vs_lft > strong')
            team2_point = li_tag.find_element_by_css_selector('div.vs_rgt > strong')

            team1_name = team1_name.text.strip()
            team2_name = team2_name.text.strip()
            team1_point = team1_point.text.strip()
            team2_point = team2_point.text.strip()
            
            # print(f't{team1_name} {team1_point} : {team2_name} {team2 point}')
             
            data_dict['date'].append(date_str)
            data_dict['team1_name'].append(team1_name)
            data_dict['team2_name'].append(team2_name)
            data_dict['team1_point'].append(team1_point)
            data_dict['team2_point'].append(team2_point)
                 

In [5]:
def getWeekData() :
    ul_tag = driver.find_element_by_css_selector('#sch_recent_scoreboard > div > div > ul')
    li_list = ul_tag.find_elements_by_css_selector('li')
    
    date_list = []
    
    # 주중 일정을 보기위한 스크립트 코드 추출
    for li_tag in li_list[1:-1] :
        a_tag = li_tag.find_element_by_css_selector('a')
        script_code = a_tag.get_attribute('onclick')
        script_code = script_code.split(':')[1]
        
        date_list.append(script_code)
    
    # 다음 주 첫번째 날짜를 보기위한 스크립트 코드 추출
    a_tag = li_list[-1].find_element_by_css_selector('a')
    script_code = a_tag.get_attribute('onclick')
    script_code = script_code.split(':')[1]
    
    return date_list, script_code


In [6]:
# getData('20190311')
driver.execute_script("scoreBoardInstance.setScoreBoard('20190311')")
time.sleep(1)

while True :
    date_list, next_week = getWeekData()

    for script_code in date_list :
        getData(script_code)

    if date_list[-1].find('20190331') == -1 :
        driver.execute_script(next_week)
        time.sleep(1)

    else :
        break
        
# 저장
df1 = pd.DataFrame.from_dict(data_dict)
df1.to_csv('baseball.csv', encoding = 'cp949', index = False)
print('수집완료')

20190311 : 경기없음
20190312 : 경기있음
20190313 : 경기있음
20190314 : 경기있음
20190315 : 경기있음
20190316 : 경기있음
20190317 : 경기있음
20190318 : 경기없음
20190319 : 경기있음
20190320 : 경기있음
20190321 : 경기없음
20190322 : 경기없음
20190323 : 경기있음
20190324 : 경기있음
20190325 : 경기없음
20190326 : 경기있음
20190327 : 경기있음
20190328 : 경기있음
20190329 : 경기있음
20190330 : 경기있음
20190331 : 경기있음
수집완료
